In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("/Users/vivek/work/GIT/MLops_survival_prediction/artifacts/raw/titanic_train.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
1,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
2,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
3,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
4,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,572.000000,712.000000,712.000000,712.000000
mean,448.234551,0.376404,2.330056,29.498846,0.553371,0.379213,32.586276
std,256.731423,0.484824,0.824584,14.500059,1.176404,0.791669,51.969529
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.750000,0.000000,2.000000,21.000000,0.000000,0.000000,7.925000
50%,453.500000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,673.500000,1.000000,3.000000,38.000000,1.000000,0.000000,30.500000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          572 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Cabin        159 non-null    object 
 11  Embarked     710 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 66.9+ KB


In [7]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            140
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          553
Embarked         2
dtype: int64

In [8]:
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])
df["Fare"] = df["Fare"].fillna(df["Fare"].median())

df['Sex'] = df["Sex"].map({
    'male' : 0,
    'female' : 1
})
df["Embarked"] = df["Embarked"].astype('category').cat.codes

In [9]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,332,0,1,"Partner, Mr. Austen",0,45.5,0,0,113043,28.5000,C124,2
1,734,0,2,"Berriman, Mr. William John",0,23.0,0,0,28425,13.0000,NaN,2
2,383,0,3,"Tikkanen, Mr. Juho",0,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,2
3,705,0,3,"Hansen, Mr. Henrik Juul",0,26.0,1,0,350025,7.8542,NaN,2
4,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",1,6.0,4,2,347082,31.2750,NaN,2


### feature enginerrring 

In [12]:
required_cols = ["SibSp", "Parch", "Cabin", "Name", "Pclass", "Fare"]
missing = [col for col in required_cols if col not in df.columns]
if missing:
    print("Missing columns:", missing)

In [17]:
df['Familysize'] = df['SibSp'] + df["Parch"] + 1
df["Isalone"] = (df['Familysize']==1).astype(int) # if familiy ==1 then he is alone
df["HasCabin"] = df["Cabin"].notnull().astype(int) # if cabin number is assgined then it is 1 else 0
df["Title"] = df["Name"].str.extract(' ([A-Za-z]+)\.', expand=False).map( 
    {
        'Mr':0, 
        'Miss':1,
        'Mrs':2,
        'Master':3,
        'Rare':4
    }
).fillna(4) # 
df['Pclass_Fare'] = df['Pclass'] * df['Fare']
df['Age_Fare'] = df['Age'] * df['Fare']

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/var/folders/xb/sshzwv1128l88chp71np05t40000gn/T/ipykernel_78574/3958931229.py:4: SyntaxWarning: invalid escape sequence '\.'
  df["Title"] = df["Name"].str.extract(' ([A-Za-z]+)\.', expand=False).map(


In [18]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Familysize,Isalone,HasCabin,Title,Pclass_Fare,Age_Fare
0,332,0,1,"Partner, Mr. Austen",0,45.5,0,0,113043,28.5000,C124,2,1,1,1,0.0,28.5000,1296.7500
1,734,0,2,"Berriman, Mr. William John",0,23.0,0,0,28425,13.0000,NaN,2,1,1,0,0.0,26.0000,299.0000
2,383,0,3,"Tikkanen, Mr. Juho",0,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,2,1,1,0,0.0,23.7750,253.6000
3,705,0,3,"Hansen, Mr. Henrik Juul",0,26.0,1,0,350025,7.8542,NaN,2,2,0,0,0.0,23.5626,204.2092
4,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",1,6.0,4,2,347082,31.2750,NaN,2,7,0,0,1.0,93.8250,187.6500


In [19]:
X = df[['Pclass','Sex', 'Age', 'Fare', 'Embarked', 'Familysize', 'Isalone',  'HasCabin', 'Title', 'Pclass_Fare', 'Age_Fare']]
y = df['Survived']


In [21]:
df["Survived"].value_counts()

Survived
0    444
1    268
Name: count, dtype: int64

### using smote for data imbalalnce

In [22]:
smote = SMOTE()

In [23]:
x_resampled, y_resamples = smote.fit_resample(X, y)

In [26]:
y_resamples.value_counts()
# we have done oversample 

Survived
0    444
1    444
Name: count, dtype: int64

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resamples, test_size=0.2, random_state=42)

In [27]:
param_distribution = {
    'n_estimators' : [100, 200, 300],
    'max_depth' : [10, 20, 30],
    'min_samples_split' : [2,5],
    'min_samples_leaf' : [1,2]
}

In [28]:
rf = RandomForestClassifier(random_state=42)

In [33]:
random_search = RandomizedSearchCV(
                                    rf, 
                                    param_distributions = param_distribution,
                                    n_iter=10,
                                    cv = 3,
                                    scoring='accuracy', 
                                    random_state=42)
random_search.fit(x_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   param_distributions={'max_depth': [10, 20, 30],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='accuracy')

In [34]:
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(x_test)
rf_accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy  : {rf_accuracy:.2f}")

Random Forest Accuracy  : 0.88
